In [1]:
from bami.basalt.community import BasaltCommunity
from common.utils import connected_topology
import logging

from simulations.settings import LocalLocations, SimulationSettings
from asyncio import ensure_future
from simulations.basalt.basic_simulation import BasicBasaltSimulation

from multiprocess import Process, Queue

settings = SimulationSettings()
settings.peers = 50
settings.duration = 50
settings.topology = connected_topology(50)

settings.location_latency_generator = LocalLocations

logging.basicConfig(format='%(message)s',
                    filename='logs.log',
                    filemode='w',
                    level=logging.INFO)


def run_simulation(queue):
    simulation = BasicBasaltSimulation(settings, {'BasaltCommunity': BasaltCommunity})
    ensure_future(simulation.run())
    simulation.loop.run_forever()

    # Output results of the simulation
    key_to_id = {node_ins.overlays[0].my_peer.mid: node_num for node_num, node_ins in simulation.nodes.items()}
    view_vals = [key_to_id[p.mid] for p in simulation.nodes[1].overlays[0].view]
    queue.put(view_vals)
    return

# Start simulations
queue = Queue()
p = Process(target=run_simulation, args=(queue,))
p.start()
p.join()

IPv8 peer discovery complete
Simulation setup took 4.190549 seconds
Starting simulation with 50 peers...
Simulation took 22.071765 seconds


In [2]:
# 2. Analyze the results
s = queue.get()
s

[38, 34, 32, 19, 33, 17, 7, 34, 8, 2, 23, 12, 15, 40, 7, 19, 40, 14, 13, 6]